# Phase 4: RAG Review Summarization


## Step 0 — Install Dependencies

In [2]:
import sys
!{sys.executable} -m pip install groq sentence-transformers faiss-cpu pandas numpy python-dotenv tqdm -q
print('✅ Dependencies installed.')


[notice] A new release of pip is available: 25.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
✅ Dependencies installed.


## Step 1 — Set Up Groq API Key
We store the key in a `.env` file so it never gets accidentally pushed to GitHub.
Your `.gitignore` from Phase 1 already excludes `.env` files.

In [ ]:
from pathlib import Path

env_path = Path('.env')

# Only create if it doesn't exist yet
if not env_path.exists():
    # Paste your Groq API key below between the quotes
    groq_key = 'YOUR_GROQ_API_KEY_HERE'
    env_path.write_text(f'GROQ_API_KEY={groq_key}\n')
    print(f'✅ .env file created')
else:
    print(f'✅ .env file already exists — skipping')

# Load the key into environment
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv('GROQ_API_KEY')
if api_key and api_key != 'YOUR_GROQ_API_KEY_HERE':
    print(f'✅ Groq API key loaded: {api_key[:8]}...{api_key[-4:]}')
else:
    print('❌ Key not set — edit .env and replace YOUR_GROQ_API_KEY_HERE with your actual key')

✅ .env file already exists — skipping
✅ Groq API key loaded: gsk_h33q...k0jq


## Step 2 — Test Groq Connection

In [4]:
from groq import Groq

groq_client = Groq(api_key=os.getenv('GROQ_API_KEY'))

# Quick test call
test_response = groq_client.chat.completions.create(
    model='llama-3.1-8b-instant',
    messages=[{'role': 'user', 'content': 'Say: Groq connection successful!'}],
    max_tokens=20
)

print(f'✅ {test_response.choices[0].message.content}')
print(f'   Model : llama3-8b-8192')

✅ It seems like you're referring to the Groq company, a semiconductor company known for its AI processors
   Model : llama3-8b-8192


## Step 3 — Load Reviews from Phase 1

In [5]:
import pandas as pd
import numpy as np

DATA_DIR = Path('data')

df_reviews = pd.read_csv(DATA_DIR / 'reviews_clean.csv')
df_reviews = df_reviews.dropna(subset=['review_text']).reset_index(drop=True)

print(f'✅ Loaded {len(df_reviews):,} reviews')
print(f'   Products covered : {df_reviews["asin"].nunique():,}')
print(f'   Avg review length: {df_reviews["review_text"].str.len().mean():.0f} chars')
df_reviews.head(3)

✅ Loaded 5,566 reviews
   Products covered : 2,003
   Avg review length: 268 chars


,asin,review_title,review_text,rating,helpful_vote,verified_purchase
0,B078NWLG1P,Five Stars,Bought this for entertaining & it's a fun piec...,5.0,0,True
1,B082P23BMP,Great sheets,I have these sheets in two colors. They are gr...,5.0,0,True
2,B00XM0M1QQ,"Love mine, have had over a year","Love mine, have had over a year. Does not stor...",5.0,0,False


## Step 4 — Chunk Reviews
Long reviews get split into smaller chunks so the retriever can pinpoint
the most relevant part of a review rather than the whole thing.
Short reviews are kept as-is.

Chunking cell

In [9]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path('data')

# Reload reviews fresh after kernel restart
df_reviews = pd.read_csv(DATA_DIR / 'reviews_clean.csv')
df_reviews = df_reviews.dropna(subset=['review_text']).reset_index(drop=True)
print(f'Reviews loaded: {len(df_reviews):,}')
print(f'Sample review text: {df_reviews["review_text"].iloc[0][:100]}')

# Simple chunking — no fancy logic, just split long reviews
review_chunks = []

for _, row in df_reviews.iterrows():
    text = str(row['review_text']).strip()
    if len(text) < 20:
        continue

    # Just take the first 250 chars — one chunk per review, keep it simple
    review_chunks.append({
        'asin'             : str(row['asin']),
        'chunk_text'       : text[:250],
        'review_title'     : str(row.get('review_title', '')),
        'rating'           : float(row.get('rating', 0)),
        'verified_purchase': bool(row.get('verified_purchase', False))
    })

print(f'\n✅ Created {len(review_chunks):,} review chunks')
print(f'Sample chunk: "{review_chunks[0]["chunk_text"]}"')

Reviews loaded: 5,566
Sample review text: Bought this for entertaining & it's a fun piece. It's smaller than I expected though.

✅ Created 5,566 review chunks
Sample chunk: "Bought this for entertaining & it's a fun piece. It's smaller than I expected though."


In [10]:
import sys
print(f'Total chunks        : {len(review_chunks):,}')
print(f'Total reviews       : {len(df_reviews):,}')
print(f'Estimated memory    : ~{len(review_chunks) * 384 * 4 / 1024 / 1024:.1f} MB for embeddings alone')

# Check available memory
import psutil
ram = psutil.virtual_memory()
print(f'Available RAM       : {ram.available / 1024 / 1024:.0f} MB')
print(f'Total RAM           : {ram.total / 1024 / 1024:.0f} MB')

Total chunks        : 5,566
Total reviews       : 5,566
Estimated memory    : ~8.2 MB for embeddings alone
Available RAM       : 9608 MB
Total RAM           : 24576 MB


## Step 5 — Embed Review Chunks + Build Review FAISS Index
Same model as Phase 2 — keeps embedding space consistent.

In [13]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
import gc
import time

print('Loading model...')
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
print('✅ Model loaded')

# Embed directly into FAISS batch by batch
EMBEDDING_DIM = 384
review_index  = faiss.IndexFlatIP(EMBEDDING_DIM)
BATCH_SIZE    = 64

chunk_texts = [c['chunk_text'] for c in review_chunks]
total       = len(chunk_texts)
start       = time.time()

print(f'Embedding {total:,} chunks...')

for i in range(0, total, BATCH_SIZE):
    batch = chunk_texts[i : i + BATCH_SIZE]
    emb   = embed_model.encode(
        batch,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=False
    ).astype('float32')
    review_index.add(emb)
    del emb
    gc.collect()

elapsed = time.time() - start
print(f'✅ Done in {elapsed:.1f}s — {review_index.ntotal:,} vectors indexed')

# Save index
review_index_path = DATA_DIR / 'reviews_faiss_index.bin'
faiss.write_index(review_index, str(review_index_path))
print(f'✅ Index saved → {review_index_path}')

# Save metadata
review_meta_path = DATA_DIR / 'reviews_metadata.json'
with open(review_meta_path, 'w') as f:
    json.dump(review_chunks, f)
print(f'✅ Metadata saved → {review_meta_path}')

del chunk_texts
gc.collect()

Loading model...


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2095.45it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Model loaded
Embedding 3,008 chunks...
✅ Done in 5.7s — 3,008 vectors indexed
✅ Index saved → data/reviews_faiss_index.bin
✅ Metadata saved → data/reviews_metadata.json


32

## Step 6 — Build Review Retrieval Function
Given a product ASIN + a query, retrieves the most relevant review chunks for that product.

In [15]:
def get_relevant_review_chunks(
    asin: str,
    query: str,
    top_k: int = 5
) -> list[dict]:
    """
    Retrieve the most query-relevant review chunks for a specific product.

    Args:
        asin  : Product ASIN to filter reviews by
        query : User query to find relevant review content
        top_k : Number of chunks to retrieve

    Returns:
        List of relevant review chunk dicts
    """
    # Get all chunk indices that belong to this product
    product_chunk_indices = [
        i for i, chunk in enumerate(review_chunks)
        if chunk['asin'] == asin
    ]

    if not product_chunk_indices:
        return []   # No reviews for this product

    # Embed the query
    query_emb = embed_model.encode(
        [query],
        normalize_embeddings=True,
        convert_to_numpy=True
    ).astype('float32')

    # Search the review index — fetch more than we need so we can filter by ASIN
    fetch_k = min(len(review_chunks), top_k * 20)
    scores, indices = review_index.search(query_emb, fetch_k)

    # Filter to only chunks belonging to this product
    product_set = set(product_chunk_indices)
    relevant = []
    for score, idx in zip(scores[0], indices[0]):
        if idx in product_set:
            chunk = review_chunks[idx].copy()
            chunk['relevance_score'] = round(float(score), 4)
            relevant.append(chunk)
        if len(relevant) >= top_k:
            break

    return relevant


print('✅ get_relevant_review_chunks() ready.')

✅ get_relevant_review_chunks() ready.


## Step 7 — Build the RAG Summarization Function
Retrieves relevant review chunks for a product then passes them to Llama 3 via Groq to generate a structured pros/cons summary.

In [16]:
NO_REVIEW_MESSAGE = 'No reviews available for this product yet.'

def summarize_product_reviews(
    product: dict,
    query: str,
    top_k_chunks: int = 5
) -> dict:
    """
    RAG pipeline: retrieve relevant review chunks → summarize with Llama 3.

    Args:
        product      : Product dict with asin, title, price, rating etc.
        query        : Original user query
        top_k_chunks : Number of review chunks to retrieve

    Returns:
        Dict with pros, cons, verdict, and review_count
    """
    asin  = product['asin']
    title = product['title']

    # ── Step 1: Retrieve relevant review chunks ────────────────
    chunks = get_relevant_review_chunks(asin, query, top_k=top_k_chunks)

    if not chunks:
        return {
            'pros'        : [],
            'cons'        : [],
            'verdict'     : NO_REVIEW_MESSAGE,
            'review_count': 0
        }

    # ── Step 2: Build context from chunks ─────────────────────
    review_context = '\n'.join([
        f"- [{'✓' if c.get('verified_purchase') else '?'} {c['rating']}★] {c['chunk_text']}"
        for c in chunks
    ])

    # ── Step 3: Call Groq Llama 3 ─────────────────────────────
    prompt = f"""You are a helpful shopping assistant. A user is looking for: "{query}"

Here are real customer review excerpts for the product "{title}":

{review_context}

Based ONLY on the reviews above, provide a concise summary. Respond in this exact JSON format:
{{
  "pros": ["pro 1", "pro 2", "pro 3"],
  "cons": ["con 1", "con 2"],
  "verdict": "One sentence verdict on whether this product suits the user's query."
}}

Rules:
- Only include pros/cons that are mentioned in the reviews
- Keep each point under 15 words
- If reviews are mostly positive, you may have fewer cons (and vice versa)
- Respond with valid JSON only, no extra text"""

    response = groq_client.chat.completions.create(
        model='llama-3.1-8b-instant',
        messages=[{'role': 'user', 'content': prompt}],
        max_tokens=300,
        temperature=0.3    # low temperature = more consistent structured output
    )

    raw = response.choices[0].message.content.strip()

    # ── Step 4: Parse JSON response ───────────────────────────
    try:
        import json, re
        # Strip markdown code fences if present
        clean = re.sub(r'```json|```', '', raw).strip()
        parsed = json.loads(clean)
        parsed['review_count'] = len(chunks)
        return parsed
    except Exception:
        # Fallback if JSON parsing fails
        return {
            'pros'        : [],
            'cons'        : [],
            'verdict'     : raw[:300],
            'review_count': len(chunks)
        }


print('✅ summarize_product_reviews() ready.')

✅ summarize_product_reviews() ready.


## Step 8 — Build the Full Display Function
Combines product info + RAG summary into one clean output.

In [17]:
def display_product_with_summary(product: dict, query: str):
    """Display a product card with its RAG-generated review summary."""
    summary = summarize_product_reviews(product, query)

    price_str = f"${product['price']:.2f}" if product.get('price') else 'N/A'

    print(f"{'─'*65}")
    print(f"📦 {product['title'][:70]}")
    print(f"   💰 {price_str}  |  ⭐ {product.get('rating', 'N/A')} ({int(product.get('rating_count', 0)):,} reviews)")
    print(f"   🔑 ASIN: {product['asin']}")
    print()

    if summary['review_count'] == 0:
        print(f"   📝 {NO_REVIEW_MESSAGE}")
    else:
        print(f"   📝 Review Summary (from {summary['review_count']} relevant excerpts):")
        print()
        if summary.get('pros'):
            print('   ✅ Pros:')
            for pro in summary['pros']:
                print(f'      • {pro}')
        if summary.get('cons'):
            print('   ❌ Cons:')
            for con in summary['cons']:
                print(f'      • {con}')
        print()
        print(f"   💬 Verdict: {summary.get('verdict', '')}")
    print()

print('✅ display_product_with_summary() ready.')

✅ display_product_with_summary() ready.


## Step 9 — Test the Full RAG Pipeline
Load some products from Phase 1 that we know have reviews, then summarize them.

In [21]:
import json

# Load products
df_products = pd.read_csv(DATA_DIR / 'products_clean.csv')

# Find ASINs that actually have chunks in our review index
asins_with_chunks = set(c['asin'] for c in review_chunks)
print(f'ASINs with review chunks : {len(asins_with_chunks):,}')

# Pick top 3 by number of reviews in our chunk set
from collections import Counter
asin_counts = Counter(c['asin'] for c in review_chunks)
top_asins   = [asin for asin, _ in asin_counts.most_common(3)]

sample_products = df_products[df_products['asin'].isin(top_asins)].to_dict(orient='records')

print(f'Selected products:')
for p in sample_products:
    count = asin_counts[p['asin']]
    print(f'  • {p["title"][:60]} ({count} chunk(s))')

# Run RAG summaries
test_query = 'compact coffee maker for small kitchen'
print(f'\n🔍 Query: "{test_query}"')
print('Generating RAG summaries...\n')

for product in sample_products:
    display_product_with_summary(product, test_query)

ASINs with review chunks : 2,003
Selected products:
  • Coffeezone Small Candy / Nuts Decoration Ideas, Small Porcel (2 chunk(s))
  • Touch Choice Single Serve Coffee Brewer - Silver Coffee Make (2 chunk(s))
  • 400 Thread Count Cotton Cal King Fitted Sheets Pastel Pink 1 (2 chunk(s))

🔍 Query: "compact coffee maker for small kitchen"
Generating RAG summaries...

─────────────────────────────────────────────────────────────────
📦 Coffeezone Small Candy / Nuts Decoration Ideas, Small Porcelain Dishes
   💰 $8.99  |  ⭐ 4.3 (1,298 reviews)
   🔑 ASIN: B078NWLG1P

   📝 No reviews available for this product yet.

─────────────────────────────────────────────────────────────────
📦 Touch Choice Single Serve Coffee Brewer - Silver Coffee Maker with Ful
   💰 $nan  |  ⭐ 3.9 (89 reviews)
   🔑 ASIN: B00XM0M1QQ

   📝 Review Summary (from 2 relevant excerpts):

   ✅ Pros:
      • Makes the best coffee
      • Blows Keurig out of the water
      • Love the product
   ❌ Cons:
      • Does not store wate

## Step 10 — Test with a Product That Has No Reviews
Verifies graceful fallback works correctly.

In [22]:
# Find a product with NO reviews
asins_without_reviews = df_products[~df_products['asin'].isin(asins_with_reviews)]
no_review_product = asins_without_reviews.iloc[0].to_dict()

print('Testing graceful fallback for product with no reviews:')
display_product_with_summary(no_review_product, test_query)

Testing graceful fallback for product with no reviews:
─────────────────────────────────────────────────────────────────
📦 Set of 4 Irish Coffee Glass Mugs Footed 10.5 oz.Thick Wall Glass For C
   💰 $24.95  |  ⭐ 4.6 (18 reviews)
   🔑 ASIN: B07R3DYMH6

   📝 No reviews available for this product yet.



## Step 11 — Wire RAG into the Full Retrieve + Rerank + Summarize Pipeline
This is the complete end-to-end ShopSense pipeline so far:
FAISS → Rerank → RAG Summary for each result.

In [23]:
# Load Phase 2 FAISS index + metadata for product retrieval
product_index = faiss.read_index(str(DATA_DIR / 'faiss_index.bin'))
with open(DATA_DIR / 'faiss_metadata.json') as f:
    product_metadata = json.load(f)

print(f'✅ Product index loaded — {product_index.ntotal:,} vectors')
print(f'✅ Product metadata loaded — {len(product_metadata):,} products')

✅ Product index loaded — 10,000 vectors
✅ Product metadata loaded — 10,000 products


In [24]:
def shopsense_search(query: str, top_k: int = 3) -> None:
    """
    Full ShopSense pipeline Phases 2 + 3 + 4:
    1. FAISS semantic retrieval (top 20)
    2. Cross-encoder reranking (top K) — skipped here, using FAISS scores for now
       (full reranker runs in Colab; Phase 5 agent will call both)
    3. RAG review summarization for each result
    """
    print(f'\n{'='*65}')
    print(f'🛍️  ShopSense Search: "{query}"')
    print(f'{'='*65}\n')

    # Stage 1 — FAISS retrieval
    query_emb = embed_model.encode(
        [query],
        normalize_embeddings=True,
        convert_to_numpy=True
    ).astype('float32')

    scores, indices = product_index.search(query_emb, 20)

    candidates = []
    for score, idx in zip(scores[0], indices[0]):
        if idx == -1:
            continue
        product = product_metadata[idx].copy()
        product['faiss_score'] = round(float(score), 4)
        candidates.append(product)

    top_results = candidates[:top_k]

    print(f'Found {len(candidates)} candidates → showing top {top_k} with review summaries:\n')

    # Stage 2 — RAG summarization for each result
    for i, product in enumerate(top_results, 1):
        print(f'Result {i} of {top_k}:')
        display_product_with_summary(product, query)


print('✅ shopsense_search() pipeline ready.')

✅ shopsense_search() pipeline ready.


In [25]:
# Run the full pipeline!
shopsense_search('non stick frying pan for everyday cooking', top_k=3)


🛍️  ShopSense Search: "non stick frying pan for everyday cooking"

Found 20 candidates → showing top 3 with review summaries:

Result 1 of 3:
─────────────────────────────────────────────────────────────────
📦 Anyfish 8 Inch Frying Pan without Lid Nonstick Induction Skillet Small
   💰 $nan  |  ⭐ 4.2 (29 reviews)
   🔑 ASIN: B0B12JWHF2

   📝 No reviews available for this product yet.

Result 2 of 3:
─────────────────────────────────────────────────────────────────
📦 MICHELANGELO Frying Pan with Lid, Nonstick 8 Inch Frying Pan with Cera
   💰 $23.99  |  ⭐ 4.4 (10,637 reviews)
   🔑 ASIN: B0B6VTWMQW

   📝 Review Summary (from 2 relevant excerpts):

   ✅ Pros:
      • Non-stick surface works well initially
      • Easy to clean
      • Lightweight and easy to handle
   ❌ Cons:
      • Non-stick surface wears off after 20 uses

   💬 Verdict: This product is suitable for everyday cooking with some limitations.

Result 3 of 3:
─────────────────────────────────────────────────────────────────
📦 

In [26]:
# Try another query
shopsense_search('gift for someone who loves cooking', top_k=3)


🛍️  ShopSense Search: "gift for someone who loves cooking"

Found 20 candidates → showing top 3 with review summaries:

Result 1 of 3:
─────────────────────────────────────────────────────────────────
📦 LITSTYLES Charcuterie Boards, Bamboo Cheese Board & Knife Set, Premium
   💰 $20.99  |  ⭐ 3.9 (26 reviews)
   🔑 ASIN: B0954CX49D

   📝 No reviews available for this product yet.

Result 2 of 3:
─────────────────────────────────────────────────────────────────
📦 Funny Fork Gifts for Women Men Kids, Cute My Mac and Cheese Fork Engra
   💰 $10.99  |  ⭐ 5.0 (11 reviews)
   🔑 ASIN: B097YDGMCP

   📝 No reviews available for this product yet.

Result 3 of 3:
─────────────────────────────────────────────────────────────────
📦 Landisun Apron Kitchen Chef Cooking Gag Gift Creative Funny Grilling B
   💰 $14.99  |  ⭐ 4.6 (154 reviews)
   🔑 ASIN: B09BVKNJYK

   📝 No reviews available for this product yet.



## Step 12 — Final Summary

In [27]:
print('=' * 55)
print('     PHASE 4 COMPLETE — SUMMARY')
print('=' * 55)
print(f'  Review chunks indexed : {review_index.ntotal:,}')
print(f'  Products with reviews : {len(asins_with_reviews):,}')
print(f'  LLM model used        : llama3-8b-8192 (Groq)')
print(f'  RAG approach          : chunk → embed → retrieve → summarize')
print()
print('  Files created:')
for fname in ['reviews_faiss_index.bin', 'reviews_metadata.json']:
    fpath = DATA_DIR / fname
    print(f'    ✅ data/{fname} ({fpath.stat().st_size/1024:.0f} KB)')
print()
print('  Pipeline so far:')
print('    Phase 2 : FAISS semantic retrieval')
print('    Phase 3 : Cross-encoder reranking (Colab)')
print('    Phase 4 : RAG review summarization ← you are here')
print()
print('  ➡️  Next: Phase 5 — LangGraph Agent (VS Code)')
print('=' * 55)

     PHASE 4 COMPLETE — SUMMARY
  Review chunks indexed : 3,008
  Products with reviews : 2,003
  LLM model used        : llama3-8b-8192 (Groq)
  RAG approach          : chunk → embed → retrieve → summarize

  Files created:
    ✅ data/reviews_faiss_index.bin (4512 KB)
    ✅ data/reviews_metadata.json (732 KB)

  Pipeline so far:
    Phase 2 : FAISS semantic retrieval
    Phase 3 : Cross-encoder reranking (Colab)
    Phase 4 : RAG review summarization ← you are here

  ➡️  Next: Phase 5 — LangGraph Agent (VS Code)
